In [ ]:
import fastText

help(fastText.FastText)

In [ ]:
%%time
model = fastText.FastText.load_model('/home/jvdzwaan/Downloads/wiki.ar/wiki.ar.bin')

In [ ]:
fastText.util.find_nearest_neighbor

In [ ]:
%%time
from gensim.models import FastText as fText

model = fText.load_fasttext_format("/home/jvdzwaan/Downloads/wiki.ar/wiki.ar.bin")

In [ ]:
def most_similar(word):
    res = model.wv.most_similar(word)
    for w, s in res:
        print u'({} {})'.format(w, s)
most_similar(u'باب')